In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv("Train_data.csv")
df.head(2)

,YearStart,YearEnd,LocationAbbr,LocationDesc,Datasource,Topic,Question,Data_Value_Type,Data_Value,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,Total,Age(years),Education,Gender,Income,Race/Ethnicity,GeoLocation,ClassID,TopicID,QuestionID,DataValueTypeID,LocationID,StratificationCategory1,Stratification1,StratificationCategoryId1,StratificationID1,Class
0,2011,2011,AL,Alabama,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,Value,32.0,30.5,33.5,7304.0,Total,NaN,NaN,NaN,NaN,NaN,"(32.84057112200048, -86.63186076199969)",OWS,OWS1,Q036,VALUE,1.0,Total,Total,OVR,OVERALL,Obesity / Weight Status
1,2011,2011,AL,Alabama,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,Value,32.3,29.9,34.7,2581.0,NaN,NaN,NaN,Male,NaN,NaN,"(32.84057112200048, -86.63186076199969)",OWS,OWS1,Q036,VALUE,1.0,Gender,Male,GEN,MALE,Obesity / Weight Status


In [4]:
df.columns

Index(['YearStart', 'YearEnd', 'LocationAbbr', 'LocationDesc', 'Datasource',
       'Topic', 'Question', 'Data_Value_Type', 'Data_Value',
       'Low_Confidence_Limit', 'High_Confidence_Limit ', 'Sample_Size',
       'Total', 'Age(years)', 'Education', 'Gender', 'Income',
       'Race/Ethnicity', 'GeoLocation', 'ClassID', 'TopicID', 'QuestionID',
       'DataValueTypeID', 'LocationID', 'StratificationCategory1',
       'Stratification1', 'StratificationCategoryId1', 'StratificationID1',
       'Class'],
      dtype='object')

In [5]:
df=df[['Data_Value_Type', 'Data_Value',
       'Low_Confidence_Limit', 'High_Confidence_Limit ', 'Sample_Size',
       'Total', 'Age(years)', 'Education', 'Gender', 'Income',
       'Race/Ethnicity', 'GeoLocation', 'ClassID', 'TopicID', 'QuestionID',
       'DataValueTypeID', 'LocationID', 'StratificationCategory1',
       'Stratification1', 'StratificationCategoryId1', 'StratificationID1',
       'Class']]

In [6]:
df.isnull().sum()

Data_Value_Type                  0
Data_Value                       0
Low_Confidence_Limit             0
High_Confidence_Limit            0
Sample_Size                      0
Total                        50544
Age(years)                   41530
Education                    45592
Gender                       49029
Income                       39123
Race/Ethnicity               42220
GeoLocation                   1057
ClassID                          0
TopicID                          0
QuestionID                       0
DataValueTypeID                  0
LocationID                       0
StratificationCategory1          0
Stratification1                  0
StratificationCategoryId1        0
StratificationID1                0
Class                            0
dtype: int64

In [7]:
df.shape

(52345, 22)

In [8]:
df=df.drop(columns=["Total", "Age(years)", "Education", "Gender", "Income", "Race/Ethnicity", "GeoLocation"], axis=1)

In [9]:
df.isnull().sum()

Data_Value_Type              0
Data_Value                   0
Low_Confidence_Limit         0
High_Confidence_Limit        0
Sample_Size                  0
ClassID                      0
TopicID                      0
QuestionID                   0
DataValueTypeID              0
LocationID                   0
StratificationCategory1      0
Stratification1              0
StratificationCategoryId1    0
StratificationID1            0
Class                        0
dtype: int64

In [10]:
df=df.dropna()

In [11]:
for col in df.columns:
  print(col, df[col].nunique())

Data_Value_Type 1
Data_Value 20801
Low_Confidence_Limit 20839
High_Confidence_Limit  20887
Sample_Size 25919
ClassID 3
TopicID 3
QuestionID 9
DataValueTypeID 1
LocationID 15946
StratificationCategory1 6
Stratification1 28
StratificationCategoryId1 6
StratificationID1 28
Class 3


In [12]:
df=df.drop(columns=["Data_Value_Type","DataValueTypeID"], axis=1)

In [13]:
df.StratificationCategoryId1

0         OVR
1         GEN
2         GEN
3         EDU
4         EDU
         ... 
52340    RACE
52341     INC
52342    RACE
52343    RACE
52344     GEN
Name: StratificationCategoryId1, Length: 52345, dtype: object

In [14]:
df.sample(2)

,Data_Value,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,ClassID,TopicID,QuestionID,LocationID,StratificationCategory1,Stratification1,StratificationCategoryId1,StratificationID1,Class
12514,39.236197,36.549869,42.117967,3584.772131,OWS,OWS1,Q037,48.000000,Education,Some college or technical school,EDU,EDUCOTEC,Obesity / Weight Status
52017,29.165010,21.857454,37.720082,296.650102,FV,FV1,Q019,65.125255,Age (years),Less than high school,EDU,EDUHS,Fruits and Vegetables


In [15]:
df=df.drop(columns=["LocationID", "ClassID", "TopicID", "QuestionID", "StratificationCategory1", "Stratification1", "StratificationCategoryId1", "StratificationID1"], axis=1)

In [16]:
df.sample(2)

,Data_Value,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,Class
39456,15.620261,11.960181,20.200352,370.199598,Obesity / Weight Status
32301,30.800000,29.900000,31.800000,34012.000000,Fruits and Vegetables


In [17]:
df2=df.copy()

In [18]:
df=pd.get_dummies(df, columns=["Class"], drop_first=True )

In [19]:
X=df.drop(columns="Data_Value", axis=1)
y=df.Data_Value

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3)

In [23]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.svm import LinearSVR
from sklearn.model_selection import cross_validate

In [24]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler().fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

In [34]:
models = [RandomForestRegressor(random_state=42),XGBRegressor(random_state=42),ExtraTreesRegressor(random_state=42),
          LinearRegression(),LinearSVR(random_state=42)]
for model in models:
    print(f"********************{model}*********************")
    cv = cross_validate(model,X_train,y_train,cv=10,scoring=['neg_root_mean_squared_error'])
    print(cv['test_neg_root_mean_squared_error'].mean())


********************RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)*********************
-0.09598633801115433
********************XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, see

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

-31.165207259710485


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [35]:
for i in range(100,1001,50):
    print(f"*****{i}******")
    lr = ExtraTreesRegressor(random_state=42,n_estimators=i)
    lr.fit(X_train,y_train)
    print(np.sqrt(mean_squared_error(y_test,lr.predict(X_test))))

*****100******
0.05381620892291932
*****150******
0.0523927203749215
*****200******
0.0522183507001144
*****250******
0.0525810133598116
*****300******
0.05212550696573964
*****350******
0.05157561886274938
*****400******
0.051710735500653096
*****450******
0.05162369169744232
*****500******
0.051338128672089804
*****550******
0.05122639579470731
*****600******
0.05129722479782784
*****650******
0.051186595452460064
*****700******
0.05106865562999183
*****750******
0.05106732701092896
*****800******
0.05094041788243109
*****850******
0.05091732018736182
*****900******
0.05092293149179796
*****950******
0.050803310788525145
*****1000******
0.05077187073207234


In [25]:
for i in range(1,31,1):
    print(f"*****{i}******")
    lr = ExtraTreesRegressor(random_state=42,n_estimators=1000,max_depth=i)
    lr.fit(X_train,y_train)
    print(np.sqrt(mean_squared_error(y_test,lr.predict(X_test))))

*****1******
6.575406118020019
*****2******
4.153583518887566
*****3******
2.64838148833703
*****4******
1.6846104424362138
*****5******
1.096257992456235
*****6******
0.7451594690515551
*****7******
0.49369572534333045
*****8******
0.3128045472806576
*****9******
0.19953289115775855
*****10******
0.13875084793337772
*****11******
0.09736391757270482
*****12******
0.07593086976967467
*****13******
0.06540810965060674
*****14******
0.060226971027100326
*****15******
0.058885279259581175
*****16******
0.05766577530988407
*****17******
0.057145786590282154
*****18******
0.05788610265467757
*****19******
0.0566819459248693
*****20******
0.05761282981428189
*****21******
0.057720261365779346
*****22******
0.05843970929017119
*****23******
0.058259661990703954
*****24******
0.05901339210872303
*****25******
0.058665703910265286
*****26******
0.058886463493703856
*****27******
0.05862265443033968
*****28******
0.058052217685859345
*****29******
0.05864697509537361
*****30******
0.058947297573

In [33]:
lr = ExtraTreesRegressor(random_state=42,n_estimators=1000,max_depth=20)
lr.fit(X_train,y_train)
print(np.sqrt(mean_squared_error(y_test,lr.predict(X_test))))

0.05761282981428189


In [34]:
test=pd.read_csv("Test_data.csv")

In [35]:
test.head(2)

,YearStart,YearEnd,LocationAbbr,LocationDesc,Datasource,Topic,Question,Data_Value_Type,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,Total,Age(years),Education,Gender,Income,Race/Ethnicity,GeoLocation,ClassID,TopicID,QuestionID,DataValueTypeID,LocationID,StratificationCategory1,Stratification1,StratificationCategoryId1,StratificationID1,Class
0,2016,2016,US,National,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,Value,34.9,35.5,438479.0,Total,NaN,NaN,NaN,NaN,NaN,NaN,OWS,OWS1,Q037,VALUE,59.0,Total,Total,OVR,OVERALL,Obesity / Weight Status
1,2016,2016,US,National,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,Value,40.6,41.5,198440.0,NaN,NaN,NaN,Male,NaN,NaN,NaN,OWS,OWS1,Q037,VALUE,59.0,Gender,Male,GEN,MALE,Obesity / Weight Status


In [36]:
test=test[['Low_Confidence_Limit', 'High_Confidence_Limit ',
       'Sample_Size', "Class"]]

In [37]:
test=pd.get_dummies(test, columns=["Class"])

In [38]:
test.head(2)

,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,Class_Obesity / Weight Status,Class_Physical Activity
0,34.9,35.5,438479.0,1,0
1,40.6,41.5,198440.0,1,0


In [39]:
test=scaler.transform(test)

In [40]:
target = lr.predict(test)

In [41]:
target

array([35.27901212, 41.12399609, 29.28159927, ..., 29.96233824,
       44.24048435, 32.63471642])

In [ ]:
#res = pd.DataFrame(target) 
#res.columns = ["prediction"]
#res.to_csv("submission.csv", index = False)